In [ ]:
!pip install SpeechRecognition moviepy
!pip install sentencepiece
!pip install transformers
import speech_recognition as sr
import time
import moviepy.editor as mp
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

     |████████████████████████████████| 32.8MB 105kB/s 
     |████████████████████████████████| 1.2MB 8.6MB/s 
     |████████████████████████████████| 2.0MB 9.0MB/s 
     |████████████████████████████████| 890kB 32.6MB/s 
     |████████████████████████████████| 3.2MB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b7048034565f9dc55a75f4ed3b5240c4281a3c09b5ae979e14e76e5be026db00
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2809856/45929032 bytes (6.1%)6045696/45929032 bytes (13.2%)9109504/45929032 bytes (19.8%)

In [ ]:

start_time = time.time()  #Start timer to calculate the  total execution time 


In [ ]:


clip = mp.VideoFileClip(r"30days.mp4") #read the clip

clip.audio.write_audiofile(r"converted.wav") #save extracted audio as audio file

[MoviePy] Writing audio in converted.wav



100%|██████████| 4571/4571 [00:02<00:00, 2093.85it/s]

[MoviePy] Done.


In [ ]:
r = sr.Recognizer()           #instance of recognizer
audio = sr.AudioFile("converted.wav")     
with audio as source: 
 audio_file = r.record(audio)             # records the data from the file to an AudioData instance
result = r.recognize_google(audio_file)   #Recognize the data
with open('recognized.txt',mode ='w') as file:  #write into a text file
   file.write("Recognized Speech:") 
   file.write("\n") 
   file.write(result) 
   print("ready!")

RequestError: ignored

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-large')  #t5-base  is a pretrained model.
tokenizer = T5Tokenizer.from_pretrained('t5-large')  
device = torch.device('cpu')   #use the  cpu for training 


In [ ]:
f = open("recognized.txt", "r")  
#print(f.read())

In [ ]:
text= f.read()

In [ ]:
preprocess_text = text.strip().replace("\n","")  #remove newlines and append space
t5_prepared_Text = "summarize: "+preprocess_text 
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text,return_tensors="pt").to(device)

original text preprocessed: 
 Recognized Speech:a few years ago I felt like I was stuck in a rut so I decided to follow in the footsteps of the Great American philosopher Morgan Spurlock and try something new for 30 days the idea is actually pretty simple think about something you've always wanted to add your life and try it for the next 30 days it turns out 30 days or more memorable to take a picture everyday and I remember exactly where I was and what I was doing that day I also noticed that if you really want something badly enough you can do anything for 30 days every November 10th of thousands of people try to write their own 50000 word novel from scratch in 30 days it turns out all you have to do is a day for a month so I did by the way the secret is not to go to sleep until you've written your words for the day you might be sleep deprived but thanks


In [ ]:
# crate summary 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)



Summarized text: 
 every year thousands of people try to write their own 50000 word novel from scratch in 30 days. recognized speech was inspired by the great american philosopher Morgan spurlock who challenged people to try something new for a month. the secret is not to go to sleep until you've written your words for the day.


In [ ]:
print("--- %s seconds ---" % (time.time() - start_time)) #calculate the execution time for our reference 

In [ ]:
#!cat /proc/cpuinfo  #System  info of collab notebook